In [1]:
from datasets import load_dataset
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

C:\Users\Akshay\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds_1 = load_dataset("lukaspetersson/wrong_math")
ds_2 = load_dataset("layoric/labeled-multiple-choice-explained")
ds_3 = load_dataset("under-tree/labeled-multiple-choice")
ds_4 = load_dataset("anaonymous-aad/GenQA_multiple_choice")

### Dataset 1

In [3]:
dataset_1 = pd.DataFrame()
agent_1 = []
agent_2 = []
instruction = ds_1["train"]["instruction"]
output = ds_1["train"]["output"]

def addition(str):
    t = str.split("+")
    return int(t[0]) + int(t[1]) 

agent_1.extend([i.replace("What","").replace("?","").replace("Answer","equals ").replace(":","").replace(" is","is") + str(output[counter])+" ?" for counter,i in enumerate(instruction)])
agent_2.extend(["No, "+i.split("?")[0].split("is ")[1]+" equals " + str(addition(i.split("?")[0].split("is ")[1])) for counter,i in enumerate(instruction)])

In [4]:
len(agent_1)

100000

### Dataset 4

In [5]:
prompt = ds_4["train"]["prompt"]
messages = ds_4["train"]["messages"]

#### Correct Ans

In [6]:
p = [];m = []
for i in tqdm(range(0,len(messages))):
    if "Explanation" not in messages[i][1]["content"]:
        continue
    else:
        p.append(prompt[i].replace("\n",""))
        m.append("Correct option is "+ messages[i][1]["content"].split("Explanation")[0].replace("\n","").replace("*","") + 
                ". <Explanation>: " + messages[i][1]["content"].split("Explanation")[1].replace("\n","").replace("*","").replace(":",""))
agent_1.extend(p)
agent_2.extend(m)

100%|██████████| 372610/372610 [00:01<00:00, 230685.16it/s]


In [7]:
len(agent_1)

472248

#### Wrong Ans

In [8]:
def random_val(li):
    return random.choice(li)
p = [];m = []
for i in tqdm(range(0,len(messages))):
    if "Explanation" not in messages[i][1]["content"]:
        continue
    else:
        try:
            options = []
            if "(A)" in prompt[i]: 
                option_string = prompt[i].replace("\n","").split("(A)")
                li = ["(B)","(C)","(D)"]
                for j in li:
                    option_string = option_string[1].split(j)
                    options.append(option_string[0])
                options.append(option_string[1])
                correct_option = messages[i][1]["content"][:3]
                idx_val = 0
                if correct_option == "(D)":
                    idx_val = 3
                elif correct_option == "(C)":
                    idx_val = 2
                elif correct_option == "(B)":
                    idx_val = 1
                options.pop(idx_val)
                p.append(prompt[i].replace("\n","") + " <Answer>: "+random_val(options))
                m.append("No the correct answer is "+ messages[i][1]["content"].split("Explanation")[0].replace("\n","").replace("*","") + 
                ". <Explanation>: " + messages[i][1]["content"].split("Explanation")[1].replace("\n","").replace("*","").replace(":",""))
            else:
                continue
        except Exception as e:
            pass
agent_1.extend(p)
agent_2.extend(m)

100%|██████████| 372610/372610 [00:02<00:00, 133063.73it/s]


In [9]:
len(agent_1)

787921

In [10]:
dataset_1["agent_1"] = agent_1
dataset_1["agent_2"] = agent_2

In [11]:
dataset_1

,agent_1,agent_2
0,is 6+9 equals 30 ?,"No, 6+9 equals 15"
1,is 10+2 equals 24 ?,"No, 10+2 equals 12"
2,is 1+7 equals 16 ?,"No, 1+7 equals 8"
3,is 3+5 equals 16 ?,"No, 3+5 equals 8"
4,is 10+4 equals 28 ?,"No, 10+4 equals 14"
...,...,...
787916,A historic building is slated for redevelopmen...,No the correct answer is (B). <Explanation>: A...
787917,A researcher is conducting a study on the effe...,No the correct answer is (A) Between-subjects ...
787918,Scientists have discovered a novel gene involv...,No the correct answer is (B). <Explanation>: T...
787919,A 25-year-old male presents with the following...,No the correct answer is A. <Explanation>: Th...
